In [1]:
import numpy as np
x = np.linspace(-200,200,5000)
deltax = x[1]-x[0]

%matplotlib tk

from matplotlib.animation import FuncAnimation
import matplotlib.pyplot as plt

In [7]:
def norm(phi):
    norm = np.sum(np.square(np.abs(phi)))*deltax
    return phi/np.sqrt(norm)


def complex_plot(x,y):
    real = np.real(y)
    imag = np.imag(y)
    a,*_ = plt.plot(x,real, ':', label='Re')
    b,*_ = plt.plot(x,imag, ':', label='Im')
    plt.xlim(-2,2)
    p,*_ = plt.plot(x,np.abs(y),label='$\sqrt{P}$')
    return a,b,p
    
def wave_packet(pos=0,mom=0,sigma=0.2):

    return (2 * np.pi * np.square(sigma))**(-1/4) * np.exp(1j*mom*x)*np.exp(-np.square((x-pos)/(2*sigma)),dtype=complex)
#     return  np.exp(1j*mom*x)





def d_dxdx(phi,x=x):
    dphi_dxdx = -2*phi
    dphi_dxdx[:-1] += phi[1:]
    dphi_dxdx[1:] += phi[:-1]
    return dphi_dxdx/np.square(deltax)

def d_dt(phi,h=1,m=1,V=0):
    return 1j*h/2/m * d_dxdx(phi) - 1j*V*phi/h


def rk4(phi, dt, **kwargs):
    k1 = d_dt(phi, **kwargs)
    k2 = d_dt(phi+dt/2*k1, **kwargs)
    k3 = d_dt(phi+dt/2*k2, **kwargs)
    k4 = d_dt(phi+dt*k3, **kwargs)
    return phi + dt/6*(k1+2*k2+2*k3+k4)


def simulate(phi_sim, 
             V=0, 
             steps=100000, 
             dt=0.005, 
             orthogonal_to = 0, 
             save_every=1000):
    simulation_steps = np.zeros((int(steps/save_every), 5000), dtype = complex)
    l = 0
    for i in range(steps):
        phi_sim = rk4(phi_sim,dt,V=V)
        if np.sum(orthogonal_to) != 0:
            phi_sim = phi_sim - np.sum(np.conjugate(orthogonal_to)*phi_sim)*deltax*orthogonal_to
        phi_sim = norm(phi_sim)
        if i % save_every == 0:
            simulation_steps[l,:] = phi_sim
            l += 1
    return simulation_steps

def box_init(V_0, a_begin, a_end):
        plt.gcf().axes[0].axvspan(a_begin, a_end, alpha=0.2, color='orange', label = r'$V_0$ = {}'.format(V_0))
        plt.xlim(-50,50)
        plt.ylim(-2,2)

        
def pot_init():
    plt.fill_between(x,(strange_potential(x)),-2,color='orange',alpha=0.2)

def animate(simulation_steps, V_0, a_begin, a_end, init_func = None):
    fig = plt.figure()
    re,im,prob = complex_plot(x,simulation_steps[0])
    plt.xlim(-50,50)
    plt.ylim(-2,2)
    plt.xlabel('Position')
    plt.ylabel(r'$\psi$')
    if init_func:
        init_func()
    
    

    def animation(frame):
        prob.set_data((x, np.abs(simulation_steps[frame])))
        re.set_data((x, np.real(simulation_steps[frame])))
        im.set_data((x, np.imag(simulation_steps[frame])))
        return prob,re,im

    anim = FuncAnimation(fig, animation, frames=int(len(simulation_steps)), init_func = pot_init(), interval = 50)
    plt.show()
    plt.legend()
    

    return anim

In [3]:
#parameters

V_0 = [1,2,3,4,5]
a_begin = 1.0
a_end = 2.0
width = a_end - a_begin
# a_end = [1.5,1.8 , 2.0, 3.0, 4.0]
# width = [x -a_begin for x in a_end]
momentum = 1.2
variance = 0.8
mass = 1
init_pos = 0

In [5]:
# animating simulation

def strange_potential(x):
    a = 0.0001
    b = 10
    return np.exp(a*np.square(x))- b*np.exp(-a*np.square(x)) + b - 1

potential = strange_potential(x)

ground_state = simulate(wave_packet(pos = init_pos, mom = momentum, sigma = variance), dt=-1e-3j, V = potential, steps=60000,save_every=200)
animate(ground_state, V_0, a_begin, a_end)

In [8]:
ground_state_sim = simulate(ground_state[-1,:], V= potential, steps = 50000, save_every = 200)
animate(ground_state_sim, V_0, a_begin, a_end)

In [11]:
phi = wave_packet(pos = init_pos, mom = momentum, sigma = variance)
phi_1 = phi - np.sum(np.conjugate(ground_state[-1,:])*phi)*deltax*ground_state[-1,:]

first_state = simulate(phi_1, dt=-1e-3j, orthogonal_to = ground_state[-1,:],V = potential, steps=80000,save_every=200)
animate(first_state, V_0, a_begin, a_end)

In [12]:
first_state_sim = simulate(first_state[-1,:], V = potential, steps = 50000, save_every = 200)
animate(first_state_sim, V_0, a_begin, a_end)